# Fitting a physical model using Gamera

## Context

[Gamera](http://libgamera.github.io/GAMERA/docs/documentation.html) is a source modelling code for gamma ray astronomy that allows for the calculation of gamma-ray spectra from underlying populations of leptonic and hadronic cosmic rays. Fitting the resulting spectra to data would directly allow to constrain the parameters of these underlying distributions.

## Proposed approach


Here, we will demonstrate the two possible ways in which Gamera can be used in combination with gammapy to fit the spectra of gamma-ray sources. As an example, we will use the scenario of a leptonic source producing gamma rays through Inverse Compton scattering.

The two possible approaches are
        
* Subclassing `~gammapy.modeling.models.SpectralModel` and creating a `GameraSpectralModel`. This is probbaly the most elegant way to combine Gamera and gammapy, but can also be slow because it requires the repeated execution of Gamera for every model evaluation
* Creating a `~gammapy.modeling.models.TemplateNDSpectralModel` from Gamera spectra produced on a grid of model parameters. While less elegant than the first approach, it can be significantly faster because no Gamera evaluations are required during the fit.

For more modelling options with Gamera, see the [documentation](http://libgamera.github.io/GAMERA/docs/documentation.html).
Also note that gamera is not pip installable. See installation instructions [here](http://libgamera.github.io/GAMERA/docs/download_installation.html).

## Setup

As always, we start the notebook with some setup and imports. The environment variable `GAMERA_LIB_PATH` should point to the direcotry containing the installed Gamera library.

In [ ]:
import sys
import os
sys.path.append(os.environ["GAMERA_LIB_PATH"])
import gappa as gp

In [ ]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord,Angle
import itertools
from regions import CircleSkyRegion

In [ ]:
from gammapy.maps import MapAxis,RegionNDMap,RegionGeom
from gammapy.modeling.models import (
    SpectralModel,
    SPECTRAL_MODEL_REGISTRY,
    TemplateNDSpectralModel
)
from gammapy.modeling import Parameter 

## Approach 1: GameraSpectralModel

Below is the implementation of the `GameraSpectralModel` class. It is a subclass of `SpectralModel`. With every evaluation, it evolves a population of electrons in a time-independent environment (i.e. fixed magnetic field and radiation field), takes the output electron spectrum and calculates the gamma-ray spectrum from this.

As the radiation field, we use the model from [Popescu et al. 2017](https://doi.org/10.1093/mnras/stx1282) together with the CMB spectrum, in this case evaluated at the position of the crab nebula. The data of the model can also be found [here](http://cdsarc.u-strasbg.fr/viz-bin/Cat?J/MNRAS/470/2539#/browse). Any other model or radiation spectrum is also possible. It is read here from a txt file with the energies and energy densities of the radiation.

The model has a number of parameters related to the modelled source. Many of these should not be fit, but just be set at the model instantiation and frozen. 

Also, Gamera does not work with astropy units. Therefore, the input quantities to the `GameraSpectralModel` need to be internally stripped of their units for the Gamera calculations. The units are then reattached to the output.

For more details about the Gamera code and methods, see [here](http://libgamera.github.io/GAMERA/docs/time_independent_modeling.html).




In [ ]:
class GameraSpectralModel(SpectralModel):
    """Spectral model from GAMERA synchrotron and IC emission.
    A power law with cutoff is assumed for the electron spectrum.
    To limit the span of the parameters space, we fit the log10 of the parameters
    whose range is expected to cover several orders of magnitudes.

    Some of the parameters (like the distance) should not be fit, but are there just to evaluate the model.
    """

    tag = ["GAMERA", "g"]
    # parameters that we actually might want to fit
    index = Parameter("index", 2.5, min=1.5, max=4.0) #it's positive: E^-index
    log10_effic = Parameter("log10_effic", -2, min=-8, max=0)
    log10_E_min = Parameter("log10_E_min", -5, min=-9, max=0,frozen=True) # in TeV
    log10_E_cut = Parameter("log10_E_cut", 4, min=0, max=6,frozen=True)  # in TeV
    log10_B = Parameter("log10_B", -5, min=-8, max=-2,frozen=True) #in Gauss

    # parameters that we should not fit but just provide
    L_tot = Parameter("L_tot", "1e43 erg s-1", min=1e30, max=1e50, frozen=True) # in ergs/s
    d = Parameter("d", "2 kpc", min=1, max=10, frozen=True) # in kpc
    age = Parameter("age", "3e4 yr", min=1e3, max=1e6, frozen=True)


    @staticmethod
    def evaluate(
        energy,
        log10_B,
        index,
        log10_effic,
        log10_E_min,
        log10_E_cut,
        L_tot,
        d,
        age,
    ):
        # conversions
        B = (10 ** log10_B) * u.G
        effic = 10 ** log10_effic
        E_min = (10 ** log10_E_min) * u.TeV
        E_cut = (10 ** log10_E_cut) * u.TeV

        # Get the relevant GAMERA modules
        fu = gp.Utils()
        fp = gp.Particles()
        fr = gp.Radiation()
        fp.ToggleQuietMode()
        fp.SetSolverMethod(1)
        fr.ToggleQuietMode()

        # GAMERA doesn't know about astropy units
        # so we need to get everything in the right unit
        # and as a normal float and not a quantity
        b_field = B.to_value('G')
        t_max = age.to_value('yr')
        distance = d.to_value('pc')
        e_cut = np.log10(E_cut.to_value('erg'))
        e_min = np.log10(E_min.to_value('erg'))
        norm = (effic * L_tot).to_value('erg s-1')
        energy_shape=energy.shape
        photon_energies = energy.flatten().to_value('erg')
        index = index.value
        # Define the electron energy range and the injected spectrum
        # We want to go beyond the cutoff so we use two orders of magnitude more than e_cut
        energy_electrons_edges = np.logspace(e_min, e_cut+2, 100+1) # it's in ergs

        padded = False

        
        # GAMERA removes the highest and lowest energy bin so we need to pad it so that our choice of E_min is the true one
        step_log = np.diff(np.log10(energy_electrons_edges))[0]
        energy_electrons_edges = np.append(energy_electrons_edges[0]*10**-step_log, energy_electrons_edges)
        energy_electrons_edges = np.append(energy_electrons_edges, energy_electrons_edges[-1]*10**step_log)
        padded = True

        if padded:
            range = slice(1,-1, 1)
        else:
            range = slice(None, None, 1)

        energy_electrons = MapAxis.from_edges(energy_electrons_edges, interp='log', unit='erg')
        exp_cutoff = np.exp(-(energy_electrons.center.to('erg')/E_cut.to('erg')).value)
        power_law = (energy_electrons.center.to_value('erg') ** -index)*exp_cutoff

        # Normalize to total power
        # Note that if we have padded, the range is different
        power_law *= norm / fu.Integrate(list(zip(energy_electrons.center.to_value('erg')[range], power_law[range] * energy_electrons.center.to_value('erg')[range])))

        # Bundle together in the way GAMERA wants
        power_law_spectrum = np.array(list(zip(energy_electrons.center.to_value('erg'), power_law)))

        #Read the radiation field spectrum, in this case the Popescu et al. 2017 model + CM at the crab nebula position. 
        rad_field = np.loadtxt("radiation_field.txt")

        RADIATION_FIELD = list(zip(rad_field[:,0], rad_field[:,1]))

        # Set everything that is left
        fp.AddArbitraryTargetPhotons(RADIATION_FIELD)
        fp.SetCustomInjectionSpectrum(power_law_spectrum)  # the injection rate is constant and given by the normalization of injected PL
        fr.AddArbitraryTargetPhotons(RADIATION_FIELD)
        fr.SetDistance(distance)
        fp.SetBField(b_field)
        fr.SetBField(b_field)


        #Run the evolution of the electron spectrum and set the output as parent population for radiation calculation
        fp.SetAge(t_max)
        fp.CalculateElectronSpectrum()
        sp = np.array(fp.GetParticleSpectrum())
        fr.SetElectrons(sp)

        # compute the gamma-ray spectrum on the points given by the data
        fr.CalculateDifferentialPhotonSpectrum(photon_energies)
        rad = np.array(fr.GetTotalSpectrum())  # this is (dN/dE vs E, units: 1/ erg / cm^2 / s vs TeV)

        sed=rad[:,1]

        # Get back to the world of units
        sed *= u.erg**-1 * u.cm**-2 *u.s **-1

        return sed.to("1 / (cm2 eV s)").reshape(energy_shape)

In [ ]:
SPECTRAL_MODEL_REGISTRY.append(GameraSpectralModel)

In [ ]:
gamera_custom_spectral_model=GameraSpectralModel()

## Approach 2: TemplateNDSpectralModel

The other option is to fill a grid of spectra dependent on the parameters to be fit. Here, we demonstrate this using the `GameraSpectralModel` to fill the grid of spectra, with `log10_effic` and `index` as the parameters to be fit. Of course, any other function that returns the spectrum as the function of the parameters to be fit could be used.

As an example we use the crab nebula position as the center of the `CircleSkyRegion` on which the model is based.

In [ ]:
def make_template_spectral_model(spectral_model,on_region,n_energies,n_efficiencies,n_index,log10_energy_bounds,log10_eff_bounds,index_bounds):

    energy_axis=MapAxis.from_energy_edges(np.logspace(log10_energy_bounds[0],log10_energy_bounds[1],n_energies+1)*u.TeV,name="energy_true")
    efficiency_axis=MapAxis(np.linspace(log10_eff_bounds[0],log10_eff_bounds[1],n_efficiencies+1),name="log10_effic",node_type="edges")
    spectral_index_axis=MapAxis(np.linspace(index_bounds[0],index_bounds[1],n_index+1),name="index",node_type="edges")

    region_geom=RegionGeom(on_region,axes=[energy_axis,efficiency_axis,spectral_index_axis])

    template_map=np.empty((n_efficiencies,n_index,n_energies))

    for (i, j) in itertools.product(range(n_efficiencies),range(n_index)):
        spectral_model.parameters["log10_effic"].value=efficiency_axis.center[i]
        spectral_model.parameters["index"].value=spectral_index_axis.center[j]
        template_map[j,i]=spectral_model(energy_axis.center)

    template_region_map=RegionNDMap(region_geom,data=template_map,unit=u.eV**-1*u.cm**-2*u.s**-1)

    template_spectral_model=TemplateNDSpectralModel(template_region_map)

    return template_spectral_model


In [ ]:
target_position = SkyCoord(ra=83.63, dec=22.01, unit="deg", frame="icrs")
on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

In [ ]:
gamera_template_spectral_model=make_template_spectral_model(gamera_custom_spectral_model,on_region,30,10,10,(-1,2),(-2.5,0),(1.5,4))